In [1]:
import pandas as pd
# import vaex
import numpy as np
import glob
import dask.dataframe as dd
import json
from sklearn.model_selection import train_test_split
import math
import csv
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import time
import _warnings
import tensorflow as tf
from tqdm import tqdm
import swifter
import argparse
import os

attributes = ['Timestamp', 'canID', 'DLC', 
                           'Data0', 'Data1', 'Data2', 
                           'Data3', 'Data4', 'Data5', 
                           'Data6', 'Data7', 'Flag']

2023-07-28 13:39:08.337991: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 13:39:08.390051: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-28 13:39:08.394986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 13:39:09.183664: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [27]:
def hex_to_int(hex_value):
    return int(hex_value, base=16)

def hex_string_to_array(hex_string):
    return ''.join(map(str, list(map(hex_to_int, hex_string))))

def complete_field(sample):
    if not isinstance(sample['Flag'], str):
        col = 'Data' + str(sample['DLC'])
        sample['Flag'] = sample[col]
        sample[col] = '00'
    return sample

def max_min(lst):
    """
    Finds the maximum and minimum values in a list.
    Returns a tuple containing the maximum and minimum values.
    """
    if len(lst) == 0:
        return None
    
    max_val = lst[0]
    min_val = lst[0]
    for val in lst:
        if val > max_val:
            max_val = val
        elif val < min_val:
            min_val = val
    
    return (max_val, min_val)

def extract_time_interval(time_list):
    """
    Calculates the differences between adjacent elements in an ascending array.
    Returns a list containing all differences.
    """
    if len(time_list) < 2:
        return None
    
    differences = []
    for i in range(len(time_list)-1):
        diff = time_list[i+1] - time_list[i]
        differences.append(diff)
    
    return max_min(differences)

def hamming_distance(list1, list2):
    if len(list1) != len(list2):
        raise ValueError("Input lists must have the same length.")
    
    distance = sum(a != b for a, b in zip(list1, list2))
    return distance

def print_all_distances(string_array):
    n = len(string_array)
    
    for i in range(n):
        for j in range(i + 1, n):
            distance = hamming_distance(string_array[i], string_array[j])
            print(f"Hamming distance between '{string_array[i]}' and '{string_array[j]}': {distance}")

def max_hamming_distance(string_array):
    n = len(string_array)
    max_distance = 0
    
    # Iterate through all possible rotations of the array
    for i in range(n):
        rotated_array = string_array[i:] + string_array[:i]
        distance = hamming_distance(string_array, rotated_array)
        
        max_distance = max(max_distance, distance)
    
    return max_distance


In [54]:
def split_data(file_name, attack_id):
    if not os.path.exists(file_name):
        print(file_name, ' does not exist!')
        return None
    
    df = pd.read_csv(file_name, header=None, names=attributes)
    print("Reading {}: done".format(file_name))
    df = df.sort_values('Timestamp', ascending=True)
    df = df.swifter.apply(complete_field, axis=1) 
    
    num_data_bytes = 8
    for x in range(num_data_bytes):
        df['Data'+str(x)] = df['Data'+str(x)].map(lambda x: int(x, 16), na_action='ignore')
        
    df['canID'] = df['canID'].apply(lambda x: hex_string_to_array(x))
    
    df = df.fillna(0)
    data_cols = ['Data{}'.format(x) for x in range(num_data_bytes)]
    df[data_cols] = df[data_cols].astype(int) 
    df['Data'] = df[data_cols].values.tolist()
    # df['Data'] = df['Data'].apply(lambda x: ''.join(map(str, x)))
    df['Flag'] = df['Flag'].apply(lambda x: True if x=='T' else False)
    all_payload_list = df['Data'].tolist()
    print(all_payload_list)
    normal_df = df[df['Flag'] == False]

    unique_canID_list = normal_df['canID'].unique().tolist()
    
    attack_name = file_name.split('/')[2].split('_')[0]
    
    for id in unique_canID_list:
        filter_df = normal_df[normal_df['canID'] == id]
        csvfile = '../data/entropy/{}_{}.csv'.format(id, attack_name)
        # filter_df.to_csv(csvfile, index=False, mode='a+')
        # print(filter_df)
        time_list = filter_df['Timestamp'].tolist()
        payload_list = filter_df['Data'].tolist()
        max_time, min_time = extract_time_interval(time_list)
        print(payload_list, " has length: ", len(payload_list))
        # print_all_distances(payload_list)
        print("Maximum Hamming Distance of the string array: ", hamming_distance('216001380000', '2168501390000'))
                            
        a
        # print("Time interval of: {}\n max: {}\n min: {}".format(id, max_time, min_time))
        
        

    # print(unique_canID_list)
    
    print("Pre-processing: Done")
    
    # print("BYTEs: ", df['Flag'][0].nbytes)
    
    # print("timestamp output", timestamp[0], " and length: ", len(timestamp[0]))
    # print("canid output", canid[0], " and length: ", len(canid[0]))
    # print("data output", data[0], " and length: ", len(data[0]))
    # print("label output", label[0], " and length: ", len(label[0]))
    
    # print('Label before use: ', df['label'])
    # df['label'] = df['label'].apply(lambda x: attack_id if any(x) else 0)
    # print('Label after use: ', df['label'])
    # print("Aggregating data: Done")
    # print('#Normal: ', df[df['label'] == 0].shape[0])
    # print('#Attack: ', df[df['label'] != 0].shape[0])
    # return df[['timestamp', 'header', 'payload', 'label']].reset_index().drop(['index'], axis=1)
 
# attack_types = ['test','DoS', 'Fuzzy', 'gear', 'RPM']
attack_types = ['Test']
indir = '../data'

for attack_id, attack in enumerate(attack_types):
    print('Attack: {} ==============='.format(attack))
    finput = '{}/{}_dataset.csv'.format(indir, attack)
    df = split_data(finput, attack_id + 1)

df

Attack: Test ===============
Reading ../data/Test_dataset.csv: done


Pandas Apply:   0%|          | 0/600 [00:00<?, ?it/s]

[[216, 0, 0, 138, 0, 0, 0, 0], [216, 0, 0, 138, 0, 0, 0, 0], [216, 0, 0, 138, 0, 0, 0, 0], [216, 0, 0, 138, 0, 0, 0, 0], [216, 0, 0, 138, 0, 0, 0, 0], [216, 0, 0, 138, 0, 0, 0, 0], [255, 127, 0, 5, 73, 0, 0, 0], [255, 127, 0, 5, 73, 0, 0, 0], [255, 127, 0, 5, 73, 0, 0, 0], [255, 127, 0, 5, 73, 0, 0, 0], [255, 127, 0, 5, 73, 0, 0, 0], [255, 127, 0, 5, 73, 0, 0, 0], [0, 0, 0, 0, 0, 1, 7, 21], [0, 0, 0, 0, 0, 1, 7, 21], [0, 0, 0, 0, 0, 1, 7, 21], [0, 0, 0, 0, 0, 1, 7, 21], [0, 0, 0, 0, 0, 1, 7, 21], [0, 0, 0, 0, 0, 1, 7, 21], [0, 33, 16, 255, 0, 255, 0, 0], [0, 33, 16, 255, 0, 255, 0, 0], [0, 33, 16, 255, 0, 255, 0, 0], [0, 33, 16, 255, 0, 255, 0, 0], [0, 33, 16, 255, 0, 255, 0, 0], [0, 33, 16, 255, 0, 255, 0, 0], [25, 128, 0, 255, 254, 127, 7, 96], [25, 128, 0, 255, 254, 127, 7, 96], [25, 128, 0, 255, 254, 127, 7, 96], [25, 128, 0, 255, 254, 127, 7, 96], [25, 128, 0, 255, 254, 127, 7, 96], [25, 128, 0, 255, 254, 127, 7, 96], [23, 128, 0, 0, 101, 127, 7, 159], [23, 128, 0, 0, 101, 127, 7,

NameError: name 'a' is not defined

In [38]:
def hamming_distance_int(list1, list2):
    if len(list1) != len(list2):
        raise ValueError("Input lists must have the same length.")
    
    distance = sum(a != b for a, b in zip(list1, list2))
    return distance

# Example usage:
list1_example = [216, 0, 0, 137, 0, 0, 0, 0]
list2_example = [216, 85, 0, 139, 0, 0, 0, 0]
distance_value = hamming_distance_int(list1_example, list2_example)
print("Hamming distance between the integer lists:", distance_value)


Hamming distance between the integer lists: 2


In [69]:
def hamming_distance_int_list(seq1, seq2):
    # Calculate the Hamming distance between two binary sequences represented as lists
    return sum(bin(x ^ y).count('1') for x, y in zip(seq1, seq2))

def all_pair_hamming_distance_int_list(sequence_list):
    n = len(sequence_list)
    hamming_distances = {}

    for i in range(n):
        for j in range(i + 1, n):
            distance = hamming_distance_int_list(sequence_list[i], sequence_list[j])
            hamming_distances[(i, j)] = distance
            # Since Hamming distance is symmetric (H(A, B) == H(B, A)),
            # we can also store the distance for (j, i) pair.
            hamming_distances[(j, i)] = distance
    
    return hamming_distances

# Example usage:
binary_sequence_list_example = [
    [216, 0, 0, 138, 0, 0, 0, 0],
    [216, 85, 0, 139, 0, 0, 0, 0],
    [216, 0, 0, 137, 0, 0, 0, 0],
    [216, 85, 0, 140, 0, 0, 0, 0],
    [216, 0, 0, 138, 0, 0, 0, 0]
]

pair_distances = all_pair_hamming_distance_int_list(binary_sequence_list_example)

# Print the Hamming distances for all pairs
for pair, distance in pair_distances.items():
    idx1, idx2 = pair
    print(f"Hamming distance between {binary_sequence_list_example[idx1]} and {binary_sequence_list_example[idx2]}: {distance}")


Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 85, 0, 139, 0, 0, 0, 0]: 5
Hamming distance between [216, 85, 0, 139, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 5
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 0, 0, 137, 0, 0, 0, 0]: 2
Hamming distance between [216, 0, 0, 137, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 2
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 85, 0, 140, 0, 0, 0, 0]: 6
Hamming distance between [216, 85, 0, 140, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 6
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 0
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 0
Hamming distance between [216, 85, 0, 139, 0, 0, 0, 0] and [216, 0, 0, 137, 0, 0, 0, 0]: 5
Hamming distance between [216, 0, 0, 137, 0, 0, 0, 0] and [216, 85, 0, 139, 0, 0, 0, 0]: 5
Hamming distance between [216, 85, 0, 139, 0, 0, 0, 0] and [216, 85, 0, 140, 0, 0, 0, 0]: 3
Ha

In [65]:
print(bin(62 ^ 0))

0b111110


In [70]:
import numpy as np

def conditional_entropy(y, x):
    # Convert input to numpy arrays if they are not already
    y = np.array(y)
    x = np.array(x)
    
    # Get unique values and their counts for X and Y
    unique_x, counts_x = np.unique(x, return_counts=True)
    unique_y, counts_y = np.unique(y, return_counts=True)
    
    # Initialize conditional entropy
    h_y_given_x = 0.0
    
    # Calculate conditional entropy
    for xi, cx in zip(unique_x, counts_x):
        # Get the subset of Y corresponding to xi in X
        y_given_xi = y[x == xi]
        
        # Get unique values and their counts for the subset of Y given xi in X
        unique_y_given_xi, counts_y_given_xi = np.unique(y_given_xi, return_counts=True)
        
        # Calculate conditional probabilities P(Y|X) for xi
        p_y_given_xi = counts_y_given_xi / cx
        
        # Calculate conditional entropy contribution for xi
        h_y_given_xi = np.sum(-p_y_given_xi * np.log2(p_y_given_xi))
        
        # Weighted sum based on P(X=xi)
        h_y_given_x += h_y_given_xi * (cx / len(x))
    
    return h_y_given_x

# Example usage:
# Assuming X and Y are two discrete random variables in the form of lists or arrays
X = [1, 2, 1, 3, 2, 3, 1, 3, 2, 1]
Y = [0, 1, 1, 0, 0, 1, 0, 1, 1, 0]

# Calculate H(Y|X)
conditional_entropy_value = conditional_entropy(Y, X)
print("Conditional Entropy H(Y|X) =", conditional_entropy_value)


Conditional Entropy H(Y|X) = 0.8754887502163469
